## Advanced Agent: Research and Report Generation

Now, let's explore a more complex agent setup involving a team of specialized agents working together to accomplish a multi-step task: researching a topic and generating a report. This approach is inspired by patterns of agentic collaboration and iterative refinement, similar to concepts seen in frameworks like LangGraph.

**Goal:** To build an orchestrator agent that manages a team of sub-agents to:
1.  **Plan:** Create an outline for a report on a given topic.
2.  **Search:** Gather information using a (mocked) Google Search tool based on the plan.
3.  **Write:** Draft the report using the plan and gathered research.
4.  **Critique:** Review the drafted report and provide feedback.

### Agent Team Architecture

Our research and report generation system will consist of the following agents, all residing in the `adk_agents/agent4_research_agent/` directory:

*   **`ResearchOrchestratorAgent` (`agent.py`)**: The main agent that manages the overall workflow. It receives the initial topic and delegates tasks to the appropriate sub-agents in sequence.
*   **`PlannerAgent` (`planner_agent.py`)**: Takes the topic from the orchestrator and generates a structured outline for the report.
*   **`SearchAgent` (`search_agent.py`)**: Receives the plan (or specific queries derived from it) and uses the `google_search` tool to find relevant information. For this demonstration, the `google_search` tool is mocked and returns predefined results for specific queries (see `tools.py`).
*   **`WriterAgent` (`writer_agent.py`)**: Uses the plan created by the `PlannerAgent` and the information gathered by the `SearchAgent` to write a first draft of the report.
*   **`CritiqueAgent` (`critique_agent.py`)**: Reviews the draft produced by the `WriterAgent` and provides constructive feedback, suggesting improvements or areas needing more work.

**Delegation Flow & Session State:**

The `ResearchOrchestratorAgent` controls the process:
1.  User provides a topic (e.g., "The Impact of Renewable Energy on Climate Change"), which the orchestrator stores in `session.state['current_topic']`.
2.  It delegates to `PlannerAgent`. `PlannerAgent` reads `session.state['current_topic']` and its output (the essay plan) is conceptually placed by the orchestrator into `session.state['essay_plan']`.
3.  It then delegates to `SearchAgent`. `SearchAgent` reads `session.state['essay_plan']` (or queries derived from it) and its findings are placed into `session.state['research_results']`.
4.  Next, `WriterAgent` is invoked. It uses `session.state['essay_plan']` and `session.state['research_results']` to create a draft, which goes into `session.state['drafted_essay']`.
5.  `CritiqueAgent` then reviews `session.state['drafted_essay']`, and its feedback is stored in `session.state['essay_critique']`.
6.  Finally, the `ResearchOrchestratorAgent` is configured with an `output_key='final_report_and_critique'`. It will combine the `drafted_essay` and `essay_critique` (or a summary of the process) into this key in the session state as its final output for the user.

This flow allows each agent to focus on its specific task, with the session state acting as the shared workspace for passing information between them.

In [ ]:
import asyncio
import importlib
import os
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types as genai_types # For creating Content/Parts if directly interacting
import logging

# Assuming the new agent files are in adk_agents.agent4_research_agent
# Make sure this path is correct based on your project structure and PYTHONPATH
try:
    from adk_agents.agent4_research_agent import agent as research_orchestrator_module
    from adk_agents.agent4_research_agent import planner_agent as planner_module
    from adk_agents.agent4_research_agent import search_agent as search_module
    from adk_agents.agent4_research_agent import writer_agent as writer_module
    from adk_agents.agent4_research_agent import critique_agent as critique_module
    from adk_agents.agent4_research_agent import tools as research_tools_module
except ImportError as e:
    print(f"ImportError: {e}. Please ensure adk_agents is in your PYTHONPATH or installed correctly.")
    print("You might need to add the parent directory of 'adk_agents' to sys.path if running from a different location.")
    # Example: 
    # import sys
    # sys.path.append('../..') # Adjust based on notebook location relative to adk_agents parent
    # from adk_agents.agent4_research_agent import agent as research_orchestrator_module

    # For the notebook to proceed, we'll define placeholder modules if imports fail
    # This is NOT a fix for the import error, but allows the notebook structure to be tested.
    class PlaceholderModule: pass
    research_orchestrator_module = planner_module = search_module = writer_module = critique_module = research_tools_module = PlaceholderModule()
    research_orchestrator_module.research_orchestrator_agent = None # or a dummy agent

# Reload modules to pick up changes if you're iterating on the agent code
if not isinstance(research_tools_module, PlaceholderModule):
    importlib.reload(research_tools_module)
    importlib.reload(planner_module)
    importlib.reload(search_module)
    importlib.reload(writer_module)
    importlib.reload(critique_module)
    importlib.reload(research_orchestrator_module)

# Basic logging configuration (set to ERROR to reduce noise, or INFO for more details)
# logging.basicConfig(level=logging.INFO) 
# print("Logging and ADK modules imported/reloaded for Research Agent.")

# Define call_agent_async if it's not defined earlier in the notebook
async def call_agent_async(query, runner, user_id, session_id):
    print(f"\nRunning agent with query: {query}\nUser ID: {user_id}, Session ID: {session_id}")
    try:
        response_generator = runner.run_agent(
            query=query,
            user_id=user_id,
            session_id=session_id,
            enable_tracing=True
        )
        async for chunk in response_generator:
            if "output" in chunk:
                print(f"{runner.agent.name} says: {chunk['output']}")
            if "tool_code" in chunk:
                print(f"{runner.agent.name} called tool: {chunk['tool_code']}")
            if "tool_result" in chunk:
                print(f"Tool returned: {chunk['tool_result']}")
    except Exception as e:
        print(f"Error during agent run: {e}")
    finally:
        print(f"Agent run completed for session {session_id}.")

### Define Constants and Session for the Research Agent

We'll use new constants for the app name, user ID, and session ID to keep this agent's interactions separate from previous examples. We also initialize the session with the research topic.

In [ ]:
APP_NAME_RESEARCH = "research_agent_app"
USER_ID_RESEARCH = "user_research_001"
SESSION_ID_RESEARCH = "session_research_001"

# Initial topic for the report
initial_research_topic = "The Impact of Renewable Energy on Climate Change"

research_session_service = InMemorySessionService()
initial_research_state = {"current_topic": initial_research_topic}

# research_session will be created by the async function below.
research_session = None

async def _create_and_print_research_session():
    global research_session # Make it accessible globally or handle its scope as needed
    print(f"Creating session {APP_NAME_RESEARCH}/{USER_ID_RESEARCH}/{SESSION_ID_RESEARCH} with state: {initial_research_state}")
    research_session = await research_session_service.create_session(
        app_name=APP_NAME_RESEARCH,
        user_id=USER_ID_RESEARCH,
        session_id=SESSION_ID_RESEARCH,
        state=initial_research_state
    )
    print(f"Initial session for research agent created. Topic: {research_session.state.get('current_topic')}")
    print(f"Session state: {research_session.state.as_dict()}")

# To run this in a notebook cell where top-level await might not be supported directly for all users:
# You would typically run: asyncio.run(_create_and_print_research_session())
# However, if you are in an environment like JupyterLab/Jupyter Notebook that has an active event loop 
# and supports %autoawait or similar, you might be able to use:
# await _create_and_print_research_session()
# For now, we'll call this in the interaction cell using asyncio.run() to ensure it's created before the agent runs.
print("Session creation function `_create_and_print_research_session` is defined.")

In [ ]:
if research_orchestrator_module.research_orchestrator_agent is not None:
    research_runner = Runner(
        agent=research_orchestrator_module.research_orchestrator_agent, # The orchestrator agent
        app_name=APP_NAME_RESEARCH,
        session_service=research_session_service
    )
    print("ResearchOrchestratorAgent runner instantiated.")
else:
    print("ResearchOrchestratorAgent could not be loaded. Runner not instantiated. Check import paths and agent definitions.")
    research_runner = None # Ensure it's defined for later cells not to break

### Running the Agent and Inspecting Results

The `ResearchOrchestratorAgent` is designed to pick up the topic from `session.state['current_topic']`. So, we can send a general instruction to start the process. We will then inspect the final session state to see the intermediate products (`essay_plan`, `research_results`, etc.) and the `final_report_and_critique`.

**Important:** The following cells that involve `await` need to be run in an environment where an asyncio event loop is already running (e.g., JupyterLab, Google Colab, or by using `asyncio.run()` explicitly if in a plain Python script being adapted for a notebook).

In [ ]:
# Cell 1: Setup and Run Session Creation (ensure this is run first)
import asyncio # ensure asyncio is imported

async def _ensure_session_is_ready():
    global research_session
    # Check if a session with the same ID already exists in the service
    existing_session = await research_session_service.get_session(
        app_name=APP_NAME_RESEARCH,
        user_id=USER_ID_RESEARCH,
        session_id=SESSION_ID_RESEARCH
    )
    if existing_session:
        print(f"Found existing session: {existing_session.session_id}")
        # Potentially update its state if needed, or just use it
        # For this demo, we'll ensure 'current_topic' is there if we reuse it
        if 'current_topic' not in existing_session.state:
            await research_session_service.update_session(existing_session.session_id, state={'current_topic': initial_research_topic})
            print(f"Updated existing session with topic: {initial_research_topic}")
        research_session = existing_session
        print(f"Using existing session. Topic: {research_session.state.get('current_topic')}")
        print(f"Session state: {research_session.state.as_dict()}")
    else:
        print("Creating new research session...")
        await _create_and_print_research_session()

# In a notebook, run this cell using await or asyncio.run(). For example:
# await _ensure_session_is_ready() 
# Or, if you don't have a running loop and are in a plain script context (less common for .ipynb directly):
# asyncio.run(_ensure_session_is_ready())

print("IMPORTANT: Run the `_ensure_session_is_ready()` async function using 'await _ensure_session_is_ready()' (if autoawait is on in Jupyter) or 'asyncio.run(_ensure_session_is_ready())' in a cell BEFORE running the agent interaction cell below.")

In [ ]:
# Cell 2: Running the agent
# Ensure the session creation cell above has been executed successfully before running this.

query_for_orchestrator = "Please generate a report based on the topic currently in the session state."
# print(f"Sending query to ResearchOrchestratorAgent: {query_for_orchestrator}")

# Assuming call_agent_async is defined (provided in an earlier cell in this example)
# and research_runner is instantiated, and research_session is initialized.

# if research_runner and research_session:
#     await call_agent_async(
#         query=query_for_orchestrator,
#         runner=research_runner,
#         user_id=USER_ID_RESEARCH,
#         session_id=SESSION_ID_RESEARCH 
#     )
# else:
#     print("Runner or session not initialized. Cannot run agent. Please check previous cells.")

print("Agent interaction cell is ready. Ensure session is created (run cell above), then uncomment and run the `await call_agent_async(...)` line after ensuring the session is created and runner is valid.")

In [ ]:
# Cell 3: Inspecting Final State

# print("\n--- Inspecting Final Session State After Research Agent Run ---")

# async def _get_and_print_final_research_state():
#     # Ensure research_session_service is accessible here
#     final_research_session = await research_session_service.get_session(
#         app_name=APP_NAME_RESEARCH, 
#         user_id=USER_ID_RESEARCH, 
#         session_id=SESSION_ID_RESEARCH
#     )
#     if final_research_session:
#         print(f"Final session state dictionary: {final_research_session.state.as_dict()}")
#         print("\n--- Key outputs from session state ---")
#         print(f"Original Topic: {final_research_session.state.get('current_topic', 'Not Set')}")
#         print(f"Essay Plan: \n{final_research_session.state.get('essay_plan', 'Not Generated')}")
#         print(f"\nResearch Results: \n{final_research_session.state.get('research_results', 'Not Generated')}")
#         print(f"\nDrafted Essay: \n{final_research_session.state.get('drafted_essay', 'Not Generated')}")
#         print(f"\nEssay Critique: \n{final_research_session.state.get('essay_critique', 'Not Generated')}")
#         print(f"\nFinal Report and Critique (from output_key): \n{final_research_session.state.get('final_report_and_critique', 'Not Generated')}")
#     else:
#         print("Error: Could not retrieve final research session state. Was the agent run?")

# To run this in a notebook (after the agent interaction cell has completed):
# await _get_and_print_final_research_state()

print("State inspection cell is ready. Uncomment and run the `await _get_and_print_final_research_state()` line after the agent interaction has completed.")

### Using the ADK Developer UI for the Research Agent

You can also interact with and debug the `ResearchOrchestratorAgent` using the ADK Developer UI:

1.  Ensure your ADK services are running.
2.  Navigate to the UI (e.g., `http://localhost:8000/ui`).
3.  Select the `research_agent_app` from the App dropdown.
4.  Enter the `User ID` (`user_research_001`) and `Session ID` (`session_research_001`).
5.  If you haven't run the session creation code from the notebook for this session ID yet (i.e., `await _ensure_session_is_ready()`), the UI will allow you to effectively create it by sending your first message. You can set the initial state (like `current_topic`) via the UI's "Update Session State" feature by providing a JSON: `{"current_topic": "Your Topic Here"}`.
6.  Send a message, like "Please generate a report on the topic in session state."
7.  Observe the agent's turn, delegation to sub-agents, tool calls (mocked search), and the final state in the UI. This provides a visual way to trace the execution flow.